v6の改良  
次元削減を使ってみる。  
やりかたは２万ランダムのデータを飛び飛びにする感じ。  
変更するのは、test_xyz,train_xyzの次元

In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import manifold
from sklearn import metrics
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import urllib.request
import shutil
import zipfile
from scipy.stats import mode

In [2]:
fold = "../train_raw_npy/"
xyz = np.load(f"{fold}acc_xyz.npy")
label = np.load(f"{fold}sampled_label.npy")
print(xyz.shape)
print(label.shape)


(196072, 3, 500)
(196072,)


xyzの形を(N,3,125)にしてみる。つまり4つのデータのうち１つのみデータを抽出。今回は平均にしてみるかな。  

In [3]:
# データを４つずつ平均して(N,3,125)に変換する
xyz_reshaped = xyz.reshape(xyz.shape[0], 3, 125,4)
xyz_mean = xyz_reshaped.mean(axis=-1)

print(xyz_mean.shape)

(196072, 3, 125)


In [5]:
xyz = xyz_mean
print(xyz.shape)
print(label.shape)

(196072, 3, 125)
(196072,)


In [6]:
# train/test split
pin = 170000
train_xyz = xyz[0:pin, :, :]
train_label = label[0:pin]
test_xyz = xyz[pin:190000, :, :]
test_label = label[pin:190000]
print(train_xyz.shape)
print(test_xyz.shape)

(170000, 3, 125)
(20000, 3, 125)


In [7]:
print(pd.Series(train_label).value_counts())
print(pd.Series(test_label).value_counts())

7.0    27641
5.0    26056
6.0    25689
8.0    22016
2.0    21185
1.0    20712
4.0    19603
3.0     7098
Name: count, dtype: int64
7.0    3612
1.0    3315
4.0    2766
6.0    2637
2.0    2489
5.0    2419
8.0    1818
3.0     944
Name: count, dtype: int64


In [8]:
# trainをランダムに20000個サンプリングする。
sample_size = 20000
indices = np.random.choice(train_xyz.shape[0], sample_size, replace=False)
train_xyz_random = train_xyz[indices]
train_label_random = train_label[indices]


In [9]:
print(train_xyz_random.shape)
print(train_label_random.shape)
print(test_xyz.shape)
print(test_label.shape)

(20000, 3, 125)
(20000,)
(20000, 3, 125)
(20000,)


In [10]:
# trainとtestのxyzを２次元配列にする
train_xyz_random_reshape = train_xyz_random.reshape(train_xyz_random.shape[0], -1)
test_xyz_reshape = test_xyz.reshape(test_xyz.shape[0], -1)
print(train_xyz_random_reshape.shape)
print(train_label_random.shape)
print(test_xyz_reshape.shape)
print(test_label.shape)
print(np.unique(train_label_random))
print(np.unique(test_label))

(20000, 375)
(20000,)
(20000, 375)
(20000,)
[1. 2. 3. 4. 5. 6. 7. 8.]
[1. 2. 3. 4. 5. 6. 7. 8.]


In [11]:
clf = BalancedRandomForestClassifier(
    n_estimators=2000,
    replacement=True,
    sampling_strategy='not minority',
    oob_score=True,
    n_jobs=6,
    random_state=42,
    verbose=1
)
clf.fit(train_xyz_random_reshape, train_label_random)
#clf.fit(sampled_x_random,sampled_label_random) # 訓練データを使ってまずランダムフォレストを学習している。

c:\Users\kaiha\anaconda3\envs\labo_Okita\lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.9s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   14.1s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   32.5s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:   58.6s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  1.5min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  2.2min
[Parallel(n_jobs=6)]: Done 2000 out of 2000 | elapsed:  2.5min finished


BalancedRandomForestClassifier(n_estimators=2000, n_jobs=6, oob_score=True,
                               random_state=42, replacement=True,
                               sampling_strategy='not minority', verbose=1)

In [12]:
Y_test_pred = clf.predict(test_xyz_reshape) # 学習器の性能をみる テストデータで
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(test_label, Y_test_pred, zero_division=0))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    1.0s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    2.3s



Classifier performance
Out of sample:
               precision    recall  f1-score   support

         1.0       0.36      0.22      0.27      3315
         2.0       0.73      0.66      0.69      2489
         3.0       0.62      0.99      0.76       944
         4.0       0.72      0.77      0.74      2766
         5.0       0.42      0.38      0.40      2419
         6.0       0.24      0.34      0.28      2637
         7.0       0.23      0.17      0.20      3612
         8.0       0.14      0.21      0.17      1818

    accuracy                           0.41     20000
   macro avg       0.43      0.47      0.44     20000
weighted avg       0.42      0.41      0.41     20000



[Parallel(n_jobs=6)]: Done 2000 out of 2000 | elapsed:    2.5s finished


# 学習データを２倍にして学習

In [13]:
# trainをランダムに40000個サンプリングする
sample_size = 40000
indices = np.random.choice(train_xyz.shape[0], sample_size, replace=False)
train_xyz_random = train_xyz[indices]
train_label_random = train_label[indices]

In [14]:
print(train_xyz_random.shape)
print(train_label_random.shape)
print(test_xyz.shape)
print(test_label.shape)

(40000, 3, 125)
(40000,)
(20000, 3, 125)
(20000,)


In [15]:
# trainとtestのxyzを２次元配列にする
train_xyz_random_reshape = train_xyz_random.reshape(train_xyz_random.shape[0], -1)
test_xyz_reshape = test_xyz.reshape(test_xyz.shape[0], -1)
print(train_xyz_random_reshape.shape)
print(train_label_random.shape)
print(test_xyz_reshape.shape)
print(test_label.shape)
print(np.unique(train_label_random))
print(np.unique(test_label))

(40000, 375)
(40000,)
(20000, 375)
(20000,)
[1. 2. 3. 4. 5. 6. 7. 8.]
[1. 2. 3. 4. 5. 6. 7. 8.]


# 学習  
アンサンブル学習訓練データ40000で学習

In [16]:
clf_2 = BalancedRandomForestClassifier(
    n_estimators=2000,
    replacement=True,
    sampling_strategy='not minority',
    oob_score=True,
    n_jobs=6,
    random_state=42,
    verbose=1
)
clf_2.fit(train_xyz_random_reshape, train_label_random)
#clf.fit(sampled_x_random,sampled_label_random) # 訓練データを使ってまずランダムフォレストを学習している。

c:\Users\kaiha\anaconda3\envs\labo_Okita\lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.4s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   29.4s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  2.0min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  4.6min
[Parallel(n_jobs=6)]: Done 2000 out of 2000 | elapsed:  5.1min finished


BalancedRandomForestClassifier(n_estimators=2000, n_jobs=6, oob_score=True,
                               random_state=42, replacement=True,
                               sampling_strategy='not minority', verbose=1)

In [17]:
Y_test_pred = clf_2.predict(test_xyz_reshape) # 学習器の性能をみる テストデータで
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(test_label, Y_test_pred, zero_division=0))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    2.2s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    3.2s



Classifier performance
Out of sample:
               precision    recall  f1-score   support

         1.0       0.33      0.21      0.26      3315
         2.0       0.74      0.70      0.72      2489
         3.0       0.71      0.99      0.83       944
         4.0       0.73      0.78      0.76      2766
         5.0       0.46      0.41      0.43      2419
         6.0       0.24      0.36      0.29      2637
         7.0       0.24      0.17      0.20      3612
         8.0       0.14      0.19      0.16      1818

    accuracy                           0.42     20000
   macro avg       0.45      0.48      0.46     20000
weighted avg       0.43      0.42      0.42     20000



[Parallel(n_jobs=6)]: Done 2000 out of 2000 | elapsed:    3.6s finished
